In [1]:
import pandas as pd
import fnmatch
import dask.dataframe as dd
from intake.source.utils import reverse_format
import os
import re
import subprocess
from tqdm.auto import tqdm
from pathlib import Path
import shutil
import numpy as np

## Create text file containing all files available

In [2]:
def get_file_list(persist_path):
    persist_path = Path(persist_path)
    persist_path.mkdir(exist_ok=True)
    root = Path("/glade/collections/cmip/CMIP6")
    dirs = [x for x in root.iterdir() if x.is_dir()]
    for directory in tqdm(dirs):
        print(directory)
        stem = directory.stem
        f = open(f"{persist_path}/{stem}.txt", "w")
        cmd = ["find", "-L", directory.as_posix(), "-name", "*.nc"]
        p = subprocess.Popen(cmd, stderr=subprocess.PIPE, stdout=f)
        p.wait()
        

In [6]:
persist_path = "./CMIP6_filelist"
#get_file_list(persist_path)


## Extract attributes of a file using information from CMI6 DRS.


References
 1. CMIP6 DRS: http://goo.gl/v1drZl
 2. Controlled Vocabularies (CVs) for use in CMIP6:
    https://github.com/WCRP-CMIP/CMIP6_CVs
    
    
Directory structure =
```<mip_era>/
    <activity_id>/
        <institution_id>/
            <source_id>/
                <experiment_id>/
                    <member_id>/
                        <table_id>/
                            <variable_id>/
                                <grid_label>/
                                    <version>
```
file name =
```<variable_id>_<table_id>_<source_id>_<experiment_id >_<member_id>_<grid_label>[_<time_range>].nc```
For time-invariant fields, the last segment (time_range) above is omitted.
Example when there is no sub-experiment: `tas_Amon_GFDL-CM4_historical_r1i1p1f1_gn_196001-199912.nc`
Example with a sub-experiment:   `pr_day_CNRM-CM6-1_dcppA-hindcast_s1960-r2i1p1f1_gn_198001-198412.nc`


In [7]:
activity_ids = list(Path(persist_path).rglob("*.txt"))
activity_ids = [activity_id.stem for activity_id in activity_ids]
activity_ids

['CFMIP',
 'CMIP',
 'LUMIP',
 'C4MIP',
 'LS3MIP',
 'OMIP',
 'HighResMIP',
 'DCPP',
 'AerChemMIP',
 'PMIP',
 'DAMIP',
 'PAMIP',
 'ScenarioMIP']

In [8]:
df = dd.read_csv(f"{persist_path}/*.txt", header=None).compute()
df.columns = ["path"]
df.head()

,path
0,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...



- 10/13/2019: 1,027,617 
- 10/15/2019: 1,113,227
- 10/16/2019: 1,129,214
- 10/16/2019: 1,138,743

In [10]:
len(df)

1138743

In [11]:
def _reverse_filename_format(file_basename, filename_template=None, gridspec_template=None):
    """
    Uses intake's ``reverse_format`` utility to reverse the string method format.
    Given format_string and resolved_string, find arguments
    that would give format_string.format(arguments) == resolved_string
    """
    try:
        return reverse_format(filename_template, file_basename)
    except ValueError:
        try:
            return reverse_format(gridspec_template, file_basename)
        except:
            print(
                f'Failed to parse file: {file_basename} using patterns: {filename_template} and {gridspec_template}'
            )
            return {}
            
def _extract_attr_with_regex(input_str, regex, strip_chars=None):
    pattern = re.compile(regex, re.IGNORECASE)
    match = re.findall(pattern, input_str)
    if match:
        match = max(match, key=len)
        if strip_chars:
            match = match.strip(strip_chars)

        else:
            match = match.strip()

        return match

    else:
        return None
    

exclude_patterns = ['*/files/*', '*/latest/*']
def _filter_func(path):
    return not any(
        fnmatch.fnmatch(path, pat=exclude_pattern) for exclude_pattern in exclude_patterns
    )



In [12]:
%%time
files = df.path.tolist()
filelist = list(filter(_filter_func, files))

CPU times: user 3.72 s, sys: 0 ns, total: 3.72 s
Wall time: 3.71 s


In [13]:
len(filelist)

722392

In [14]:
def get_attrs(filepath):
    basename = os.path.basename(filepath)
    dirname = os.path.dirname(filepath)
    filename_template = '{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_label}_{time_range}.nc'

    gridspec_template = (
                '{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_label}.nc'
            )
    
    f = _reverse_filename_format(
            basename, filename_template=filename_template, gridspec_template=gridspec_template
        )

    fileparts = {}
    fileparts.update(f)
    parent = os.path.dirname(filepath).strip('/')
    parent_split = parent.split(f"/{fileparts['source_id']}/")
    part_1 = parent_split[0].strip('/').split('/')
    grid_label = parent.split(f"/{fileparts['variable_id']}/")[1].strip('/').split('/')[0]
    fileparts['grid_label'] = grid_label
    fileparts['activity_id'] = part_1[-2]
    fileparts['institution_id'] = part_1[-1]
    version_regex = r'v\d{4}\d{2}\d{2}|v\d{1}'
    version = _extract_attr_with_regex(parent, regex=version_regex) or 'v0'
    fileparts['version'] = version
    fileparts['path'] = filepath
    return fileparts 

In [15]:
%%time
entries = list(map(get_attrs, filelist))

CPU times: user 18 s, sys: 851 ms, total: 18.8 s
Wall time: 18.8 s


In [16]:
entries[0]

{'variable_id': 'pr',
 'table_id': 'day',
 'source_id': 'BCC-ESM1',
 'experiment_id': 'ssp370',
 'member_id': 'r2i1p1f1',
 'grid_label': 'gn',
 'time_range': '20150101-20551231',
 'activity_id': 'AerChemMIP',
 'institution_id': 'BCC',
 'version': 'v20190702',
 'path': '/glade/collections/cmip/CMIP6/AerChemMIP/BCC/BCC-ESM1/ssp370/r2i1p1f1/day/pr/gn/v20190702/pr/pr_day_BCC-ESM1_ssp370_r2i1p1f1_gn_20150101-20551231.nc'}

In [17]:
len(entries)

722392

In [18]:
df1 = pd.DataFrame(entries)
df1.head()

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path
0,pr,day,BCC-ESM1,ssp370,r2i1p1f1,gn,20150101-20551231,AerChemMIP,BCC,v20190702,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,hfls,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,prsn,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,va,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,tas,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


In [19]:
len(df1)

722392

In [20]:
# Some entries are invalid
invalids = df1[~df1.activity_id.isin(activity_ids)]
df = df1[df1.activity_id.isin(activity_ids)]
invalids

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path
583241,basin,Ofx,abrupt-4xCO2,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
583243,deptho,Ofx,abrupt-4xCO2,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
583244,volcello,Ofx,abrupt-4xCO2,NorESM2-LM,r1i1p1f1,gr,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
583531,deptho,Ofx,historical,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
584584,sftof,Ofx,piControl,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
584587,deptho,Ofx,piControl,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
586036,sftof,Ofx,1pctCO2,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
586039,basin,Ofx,1pctCO2,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
586041,deptho,Ofx,1pctCO2,NorESM2-LM,r1i1p1f1,gn,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...
586042,volcello,Ofx,1pctCO2,NorESM2-LM,r1i1p1f1,gr,NaN,NCC,NorESM2-LM,v20190815,/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2...


In [24]:
invalids.path.tolist()

## Keep latest version

['/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/abrupt-4xCO2/r1i1p1f1/Ofx/basin/gn/v20190815/basin/basin_Ofx_abrupt-4xCO2_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/abrupt-4xCO2/r1i1p1f1/Ofx/deptho/gn/v20190815/deptho/deptho_Ofx_abrupt-4xCO2_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/abrupt-4xCO2/r1i1p1f1/Ofx/volcello/gr/v20190815/volcello/volcello_Ofx_abrupt-4xCO2_NorESM2-LM_r1i1p1f1_gr.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/historical/r1i1p1f1/Ofx/deptho/gn/v20190815/deptho/deptho_Ofx_historical_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/piControl/r1i1p1f1/Ofx/sftof/gn/v20190815/sftof/sftof_Ofx_piControl_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/piControl/r1i1p1f1/Ofx/deptho/gn/v20190815/deptho/deptho_Ofx_piControl_NorESM2-LM_r1i1p1f1_gn.nc',
 '/glade/collections/cmip/CMIP6/CMIP/NCC/NorESM2-LM/1pctCO2/r1i1p1f1/Ofx/s

In [25]:
grpby = list(set(df.columns.tolist()) - {'path', 'version'})
groups = df.groupby(grpby)

In [26]:
%%time
idx_to_remove = []
for _, group in groups:
    if group.version.nunique() > 1:
        idx_to_remove.extend(group.sort_values(by=['version'], ascending=False).index[1:].values.tolist())

CPU times: user 3min 36s, sys: 2.86 s, total: 3min 39s
Wall time: 3min 37s


In [27]:
len(idx_to_remove)

23653

In [28]:
len(df)

722377

In [29]:
df = df.drop(index=idx_to_remove)
len(df)

698724

In [30]:
df["dcpp_init_year"] = df.member_id.map(lambda x: float(x.split("-")[0][1:] if x.startswith("s") else np.nan))
df["member_id"] = df["member_id"].map(lambda x: x.split("-")[-1] if x.startswith("s") else x)
df.head()

,variable_id,table_id,source_id,experiment_id,member_id,grid_label,time_range,activity_id,institution_id,version,path,dcpp_init_year
0,pr,day,BCC-ESM1,ssp370,r2i1p1f1,gn,20150101-20551231,AerChemMIP,BCC,v20190702,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN
1,hfls,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN
2,prsn,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN
3,va,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN
4,tas,Amon,BCC-ESM1,ssp370,r2i1p1f1,gn,201501-205512,AerChemMIP,BCC,v20190624,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...,NaN


In [31]:
columns = ["activity_id", "institution_id", "source_id", "experiment_id", "member_id", "table_id", "variable_id",
           "grid_label", "dcpp_init_year", "version", "time_range", "path"]
df = df[columns]
df.head()

,activity_id,institution_id,source_id,experiment_id,member_id,table_id,variable_id,grid_label,dcpp_init_year,version,time_range,path
0,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,day,pr,gn,NaN,v20190702,20150101-20551231,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
1,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,hfls,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
2,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,prsn,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
3,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,va,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...
4,AerChemMIP,BCC,BCC-ESM1,ssp370,r2i1p1f1,Amon,tas,gn,NaN,v20190624,201501-205512,/glade/collections/cmip/CMIP6/AerChemMIP/BCC/B...


In [32]:
df.to_csv("../catalogs/glade-cmip6.csv.gz", compression="gzip", index=False)